![](https://miro.medium.com/max/691/0*xXUYOs5MWWenxoNz)

# Знакомство с PyTorch. Часть 3

### Сверточные нейронные сети в PyTorch  

Полносвязные нейронные сети могут выступать в качестве средства для решения задачи классификации изображений, однако на данный момент доминирующей архитектурой для решения этой задачи являются **сверточные нейронные сети (CNN)**  

Сверточные нейронные сети в PyTorch строятся по аналогии с полносвязными, но с некоторыми отличиями  
В первую очередь, для построения сверточных нейронных сетей в PyTorch применяют модуль `nn.Sequential()` для построения цепочки слоев  

В PyTorch реализовано все необходимое для построения сверточных нейронных сетей. 
Их можно представить в виде стека последовательно идущих друг за другом сверточных слоев и слоев субдискретизации (пулинга) с полносвязной сетью в конце

Сверточный слой (в двумерном случае) представлен классом `nn.Conv2D`  
Перечислим основные аргументы:
* `in_features` -- количество входных каналов изображения
* `out_features` -- количество каналов, создаваемых сверточным слоем
* `kernel_size` -- размер ядра свертки
* `stride` -- шаг свертки (по умолчанию 1
* `padding` -- определяет заполнение нулями (по умолчанию 0)

Слой maxpooling (в двумерном случае) представлен классом `nn.MaxPool2D`  
Основные аргументы:
* `kernel_size` -- размер окна пулинга
* `stride` -- шаг окна (по умолчанию равен kernel_size`  

Также существуют вариации слоя пулинга, например, `nn.AvgPool`, `nn.AdaptiveMaxPool`, `nn.AdaptiveAvgPool`

Помимо вышеперечисленных слоев при построении сверточных нейронных сетей в PyTorch часто применяют так называемый слой дропаут (Dropout). Принцип его работы крайне прост: он обнуляет в процессе обучения некоторый случайный набор параметров сети.  Выяснено, что применение этого слоя положительно влияет на склонность сверточной нейросети к обобщению данных  
Слой дропаута в PyTorch представлен классом `nn.Dropout`, который принимает единственный аргумент `p` -- долю входного тензора, который будет подвержен обнулению

Построим для решения задачи классификации изображений одежды из датасета Fashion-MNIST сверточную нейронную сеть

In [ ]:
# необходимые импорты
import torch
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

# создаем объект tranfrorm для трансформации изображений 
transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize((0.1307,), (0.3081,))
])

# загрузка тренировочного набора данных Fashion-MNIST 
train_data = dataset.FashionMNIST(r"/content/FashionMNIST",
                                  train=True,
                                  download=True,
                                  transform=transform,
                                  )

# загрузка тестового набора данных Fashion-MNIST
test_data = dataset.FashionMNIST(r"content/FashionMNIST", 
                                 train=False,
                                 download=True,
                                 transform=transform
                                 )

image, target = next(iter(train_data))
print(image.shape)

Extracting /content/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw


Extracting /content/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw



Extracting /content/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw


Extracting /content/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting content/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to content/FashionMNIST/FashionMNIST/raw


Extracting content/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to content/FashionMNIST/FashionMNIST/raw





Extracting content/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to content/FashionMNIST/FashionMNIST/raw


Extracting content/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to content/FashionMNIST/FashionMNIST/raw
Processing...
Done!
torch.Size([1, 28, 28])



In [ ]:
# создание загрузчика для тренировочного набора данных
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

# создание загрузчика для тестового набора данных
test_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [ ]:
# построение сверточной нейронной сети на PyTorch

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # стек сверточных слоев
    self.conv_layers = nn.Sequential(
        # здесь определяются сверточные слои
        # можно явно вычислить размер выходной карты признаков каждого
        # сверточного слоя по следующей формуле:
        # [(shape + 2*padding - kernel_size) / stride] + 1
        nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3, padding=1, stride=1), # (N, 1, 28, 28) 
        nn.ReLU(),
        # после первого сверточного слоя размер выходной карты признаков равен:
        # [(28 + 2*1 - 3)/1] + 1 = 28.
        nn.MaxPool2d(kernel_size=2), 
        # при прохождении слоя MaxPooling с размером окна 2
        # карты признаков сжимаются вдвое
        # 28 / 2 = 14
        nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, padding=1, stride=1),
        nn.ReLU(),
        # после второго сверточного слоя размер выходной карты признаков равен:
        #[(14 + 2*1 - 3)/1] + 1 = 14
        nn.MaxPool2d(kernel_size=2),
        # после второго слоя MaxPooling2D выходнае карты признаков имеют размерность
        # 14 / 2 = 7
    )
    # стек полносвязных слоев
    self.linear_layers = nn.Sequential(
        # после второго сверточного слоя имеем количество выходных карт признаков
        # равное 24 размером 7х7
        # эти данные и будут входными признаками в первом линейном слое
        nn.Linear(in_features=24*7*7, out_features=64),
        nn.ReLU(),
        nn.Dropout(0.2), # обнуляем 50% входного тензора для предотвращения переобучения
        nn.Linear(in_features=64, out_features=10) # количество выходных признаков равно количеству классов

    )
  
  # определение метода для прчмого распространения сигналов по сети
  def forward(self, x):
    x = self.conv_layers(x)
    # перед отправкой в блок полносвязных слоев признаки необходимо сделать одномерными
    x = x.view(x.size(0), -1)
    x = self.linear_layers(x)
    return x

# вывод структуры модели
cnn = CNN()
print(cnn)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=1176, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [ ]:
# функция, отвечающая за обучение сети

def fit(model, 
          optimizer, 
          loss_function, 
          train_loader, 
          test_loader, 
          epochs, 
          device,
         ):
    # определяем количество батчей в тренировочной выборке
    total_step = len(train_loader)
    
    
    # пускаем цикл по эпохам
    for epoch in range(epochs):
        train_loss = 0
        # для каждого батча в тренировочном наборе
        for i, batch in enumerate(train_loader):  
            # извлекаем изображения и их метки
            images, labels = batch
            # отправляем их на устройство
            images = images.to(device)
            labels = labels.to(device)
            # вычисляем выходы сети
            outputs = model(images)
            # вычисляем потери на батче
            loss = loss_function(outputs, labels)
            # обнуляем значения градиентов
            optimizer.zero_grad()
            # вычисляем значения градиентов на батче
            loss.backward()
            # корректируем веса
            optimizer.step()
            
            # корректируем значение потерь на эпохе
            train_loss += loss.item()
            
            # логируем
            if (i+1) % 500 == 0:
                print ('Эпоха [{}/{}], Шаг [{}/{}], Тренировочные потери: {:.4f}' 
                       .format(epoch+1, 5, i+1, total_step, loss.data.item()))
                
        
    # режим тестирования модели
    # для тестирования вычислять градиенты не обязательно, поэтому оборачиваем код
    # для теста в блок with torch.no_grad()
    with torch.no_grad():
        # заводим начальные значения корректно распознанных примеров и общего количества примеров
        correct = 0
        total = 0
        # для каждого батча в тестовой выборкй
        for batch in test_loader:
            # извлекаем изображения и метки
            images, labels = batch
            # помещаем их на устройство
            images = images.to(device)
            labels = labels.to(device)
            # вычисление предсказаний сети
            outputs = model(images)
            # создание тензора предсказаний сети
            _, predicted = torch.max(outputs.data, 1)
            # корректировка общего значения примеров на величину батча
            total += labels.size(0)
            # корректировка значения верно классифицированных примеров
            correct += (predicted == labels).sum().item()
        
        # логирование
        print('Точность на тестовом наборе {} %'.format(100 * correct / total))

In [ ]:
# определим функцию оптимизации
optimizer = optim.Adam(cnn.parameters(), lr=0.0005)

# определим функцию потерь
loss_function = nn.CrossEntropyLoss()

# определим устройство, на котором будет идти обучение
device = None
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

# перемещение модели на устройство
# cnn.to(device)

cuda


In [ ]:
# обучение сверточной сети на наборе данных Fashion-MNIST
epochs=5

fit(cnn,
      optimizer,
      loss_function,
      train_loader,
      test_loader,
      epochs,
      device)

Эпоха [1/5], Шаг [500/3750], Тренировочные потери: 0.2086
Эпоха [1/5], Шаг [1000/3750], Тренировочные потери: 0.0112
Эпоха [1/5], Шаг [1500/3750], Тренировочные потери: 0.2297
Эпоха [1/5], Шаг [2000/3750], Тренировочные потери: 0.0061
Эпоха [1/5], Шаг [2500/3750], Тренировочные потери: 0.2623
Эпоха [1/5], Шаг [3000/3750], Тренировочные потери: 0.1210
Эпоха [1/5], Шаг [3500/3750], Тренировочные потери: 0.0405
Эпоха [2/5], Шаг [500/3750], Тренировочные потери: 0.0085
Эпоха [2/5], Шаг [1000/3750], Тренировочные потери: 0.1839
Эпоха [2/5], Шаг [1500/3750], Тренировочные потери: 0.0928
Эпоха [2/5], Шаг [2000/3750], Тренировочные потери: 0.0098
Эпоха [2/5], Шаг [2500/3750], Тренировочные потери: 0.2070
Эпоха [2/5], Шаг [3000/3750], Тренировочные потери: 0.0725
Эпоха [2/5], Шаг [3500/3750], Тренировочные потери: 0.0558
Эпоха [3/5], Шаг [500/3750], Тренировочные потери: 0.0048
Эпоха [3/5], Шаг [1000/3750], Тренировочные потери: 0.1841
Эпоха [3/5], Шаг [1500/3750], Тренировочные потери: 0.2055


In [ ]:
# использование обученной сверочной нейронной сети для классификаций изображений одежды

# список классов
labels = train_data.classes

# возьмем пример из тестового набора
image, label = next(iter(test_data))
image = image.unsqueeze(0)

# формируем предсказания
predictions = cnn(image)
prediction = predictions.argmax()
print(predictions)
print("Предсказание: ", labels[prediction])
print("Метка:", labels[label])

RuntimeError: ignored

### История CNN. Использование предобученных CNN в PyTorch

Модели сверточных нейронных сетей существуют уже несколько десятилетий (к примеру, сеть LeNet, одна из первых сверточных архитектур, использовалась еще в 1990-х годах)  

Повсеместное их использование для решения задач компьютерного зрения началось после 2010 года, когда, во-первых, высокопроизводительные GPU стали более доступными и, во-вторых, был подстегнут очередной всплеск интереса к нейронным сетям  

Перечислим основные архитектуры CNN, которые оставили след в истории глубокого обучения  
* **AlexNet** -- сверточная архитектура, в каком-то смысле стала отправной точкой для всплекса интереса к CNN. Эта архитектура вобрала в себя все то, что ныне используется повсеместно для построения CNN: слои MaxPool и Dropout, функция активации ReLU. Это была одной из первых архитектур, продемонстрировавших эффективность обучения многослойных неросетей на GPU. Также она является победителем 2012 года в конкурсе по классификации изображений ImageNet с ошибкой в 15.3%  

![](https://www.mdpi.com/remotesensing/remotesensing-09-00848/article_deploy/html/images/remotesensing-09-00848-g001.png)

* **Inception/GoogLeNet** -- сверточная архитектура, являющаяся победителем конкурса ImageNet в 2014 году. Она является результатом устранения ошибок архитектуры AlexNet. Основное новвоведение -- модуль Inception. Он представляет собой серию разноразмерных сверток, результаты которых соединяются воедино на выходе. Исходная архитектура GoogLeNet состоит из 9 таких модулей. Она обеспечивала производительность с ошибкой 6.67%  
* **VGG** -- семйство сверточных архитектур, занявших второе место на конкурсе ImageNet. В отличие от GoogLeNet, она более проста, и представляет собой простой стек сверточных слоев. Тем не менее, в различных конфигурациях она дает на наборе данных ImageNet величину ошибки, сравнимую с GoogLeNet (около 8%).  Недостатком архитектуры VGG является то, что конечные полносвязные слои очень сильно "раздувают" сеть. Для сравнения: GoogLeNet имеет 7 миллионов параметров, когда как сети VGG могут иметь вплоть до 140 миллионов параметров.  Ее достоинством же является простота конструкции, благодаря которой она все еще остается популярной  

![](https://miro.medium.com/max/2800/0*rbWRzjKvoGt9W3Mf.png)

* **ResNet** -- семейство сверточных архитектур от Microsoft, которые выиграли конкурс ImageNet 2015, достигнув показателя ошибки классификации всего в 4.49% (вариант ResNet-152), что лучше возможностей человека. Семейство ResNet предлагало усовершенствованный подход к уплотнению слоев по сравнению с Inception, плюсом добавлялась связь от начала блока сверток непосредственно е концу. Это позволяет сигналу проходить более "глубоко" по сети, чем это возможно в VGG, при этом она остается легковесной.  

![](https://www.researchgate.net/publication/322621180/figure/fig2/AS:584852684410885@1516451154473/The-representation-of-model-architecture-image-for-ResNet-152-VGG-19-and-two-layered.png)

Помимо вышеуказанных архитектур можно отметить также более современные, например, MobileNet, SqueezeNet, NASNet, EfficientNet и т. д.




Многие из вышеперечисленных моделей представлены в PyTorch "из коробки" в модуле `torchvision.models`  



In [ ]:
import torchvision.models as models

# получим для начала модель AlexNet
alexnet = models.AlexNet(num_classes=10)
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
# ...а затем взглянем на VGG16
vgg16 = models.vgg16()
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# ...и, наконец, на ResNet152
resnet152 = models.resnet152()
print(resnet152)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Многие из вышеперечисленных моделей доступны с предварительно обученными весовыми параметрами (например, на ImageNet). что открывает широкие возможности для их использования  

Например, их можно сразу использовать по прямому назначению, т. е. использовать для классификации изображений  

In [ ]:
from PIL import Image

# возьмем для примера предобученную на наборе данных ImageNet модель ResNet152
resnet152 = models.resnet152(pretrained=True)

# создадим новый объект transform
transform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# откроем пример изображения, которого необходимо классифицировать
image = Image.open("/content/1.jpg")
image = transform(image)
image = image.unsqueeze(0)

# переводим модель в режим теста и формируем предсказания
resnet152.eval()
preds = resnet152(image)
pred = preds.argmax()
print(pred)



Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth



tensor(404)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

imagenet_classes_file = r"/content/drive/MyDrive/Colab Notebooks/imagenet_class_index.json"
with open(imagenet_classes_file) as f:
  labels = json.load(f)

# название предсказанного класса
print(labels[str(int(pred))])


['n02690373', 'airliner']


Также с помощью предобученнных моделей на PyTorch можно выполнять такую высокоэффективную методику, как **перенос обучения**  

Она заключается в использовании предобученных весов для обучения модели на собственном наборе данных  

Это оправдано тем, что, например, архитектура, обученная на ImageNet, уже очень много знает об изображениях, и в этом случае обучение на собственном наборе данных потребует меньше времени и меньше данных  

Попробуем запустить обучение модели ResNet50 на собственном небольшом наборе данных для классификации кошек и собак

In [ ]:
# приготовим данные

!unzip /content/drive/MyDrive/trash/Dogs_vs_Cats.zip

Archive:  /content/drive/MyDrive/trash/Dogs_vs_Cats.zip
   creating: Dogs vs Cats/
   creating: Dogs vs Cats/test/
   creating: Dogs vs Cats/test/cats/
  inflating: Dogs vs Cats/test/cats/cat.1500.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1501.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1502.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1503.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1504.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1505.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1506.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1507.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1508.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1509.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1510.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1511.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1512.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1513.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1514.jpg  
  inflating: Dogs vs Cats/test/cats/cat.1515.jpg  
  inflating: Dogs vs Cats/test/c

In [ ]:
train_path = r"/content/Dogs vs Cats/train"
test_path  = r"/content/Dogs vs Cats/test"

train_data = dataset.ImageFolder(train_path, transform)
test_data = dataset.ImageFolder(test_path, transform)

print(type(train_data))
print(type(test_data))

print(train_data.classes)
print(test_data.classes)

train_loader_1 = DataLoader(train_data, batch_size=16, shuffle=True)
test_loader_1  = DataLoader(train_data, batch_size=16, shuffle=True)

torchvision.datasets.folder.ImageFolder
torchvision.datasets.folder.ImageFolder
['cats', 'dogs']
['cats', 'dogs']


Предобученная модель ResNet50 уже содержит в себе много информации для распознавания и классификации изображений. Поэтому вместо переобучения можно немного изменить ее архитектуру, подстроив под текущий набор данных: заменить блок классификации

В этом случае будет происходить **заморозка** всех слоев слоев и **обучение** нового блока классификации

In [ ]:
# инициализируем предобученную модель ResNet50
pretrained_resnet50 = models.resnet50(pretrained=True)

# замораживаем слои, используя метод requires_grad()
# в этом случае не вычисляются градиенты для слоев
# сделать это надо для всех параметеров сети
for name, param in pretrained_resnet50.named_parameters():
  param.requires_grad = False


# к различным блокам модели в PyTorch легко получить доступ
# заменим блок классификатора на свой, подходящий для решения
# задачи классификации кошек и собак
pretrained_resnet50.fc = nn.Sequential(
    nn.Linear(pretrained_resnet50.fc.in_features, 500),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(500, 2)
)

# выведем модель
print(pretrained_resnet50)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [ ]:
# попробуем обучить!

epochs = 5
optimizer = optim.Adam(pretrained_resnet50.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

fit(pretrained_resnet50,
    optimizer,
    loss_function,
    train_loader_1,
    test_loader_1,
    epochs,
    device="cpu")

Точность на тестовом наборе 88.69047619047619 %


Существуют методики для улучшения показателей точности модели: аугментация данных, применение подхода дифференцированного обучения и т. д.